In [1]:
import csv
import json
import re
import collections
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

file = open('all_ubic_click_action_1013.txt',mode='r').read()
file_json = json.loads(file)

columns = list(file_json[0].keys()) + ['actionType_score']

score = {
    'cart-create': 3 ,
    'order-create': 5,
    'click': 2 ,
    'hover': 1,
    'order-cancel': -6,
    'cart-cancel': -4,

    'cart-modify-plus': 3,
    'cart-modify-minus': -3,
}

columns_value_list = []

for row in file_json:
    
    # 왼쪽부터 순차적으로 검사하나보다
    if str(row['userId']).isdigit() and str(row['productId']).isdigit() and 'categoryId' in row and str(row['categoryId']).isdigit():
        row['userId'] = int(row['userId'])
        row['productId'] = int(row['productId'])
        row['categoryId'] = int(row['categoryId'])
        
        if row['actionType'] in score:
            row['actionType_score'] = score[row['actionType']]
        else:
            row['actionType_score'] = 0

        columns_value_list.append(list(row.values()))

base_df = pd.DataFrame(columns_value_list, columns=columns)

w_actionType_df = base_df[['userId', 'productId', 'actionType_score']]

actionType_matrix_itemCF = w_actionType_df.pivot_table('actionType_score', index='userId', columns='productId')
actionType_matrix_itemCF = actionType_matrix_itemCF.fillna(0)

actionType_matrix_userCF = w_actionType_df.pivot_table('actionType_score', index='productId', columns='userId')
actionType_matrix_userCF = actionType_matrix_userCF.fillna(0)



In [2]:
# itemId - itemId list
item_similarity = cosine_similarity(actionType_matrix_userCF)
item_sim_df = pd.DataFrame(data=item_similarity, index=actionType_matrix_userCF.index, columns=actionType_matrix_userCF.index) # 행으로 연산하는거 맞다
# 루프 돌면서 : itemId
list(item_sim_df[106].sort_values(ascending=False)[1:1+4].index)

[107, 108, 2123, 3723]

In [4]:
# userId - userId[0] - itemId
# 루프 돌면서 : userId
sim_userId = actionType_matrix_userCF.corrwith(actionType_matrix_userCF[1011]).sort_values(ascending=False).index[1]
mid_itemId = actionType_matrix_userCF[sim_userId].sort_values(ascending=False).index[0]

# 루프 돌면서 : itemId
list(item_sim_df[mid_itemId].sort_values(ascending=False)[1:1+4].index)

[109, 116, 125, 131]